In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcardfraud/creditcard.csv


In [2]:
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve , auc , classification_report 
from sklearn.metrics import make_scorer, average_precision_score
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import SMOTE 

In [3]:
path = '/kaggle/input/creditcardfraud/creditcard.csv'


In [4]:

credit_card = pd.read_csv(path)
X= credit_card.drop("Class", axis = 1)
y = credit_card["Class"]

In [5]:
X_train , X_test, y_train , y_test = train_test_split(X,y, test_size=0.4, random_state = 42)


In [6]:
smote = SMOTE(sampling_strategy = {1: 20000})
X_resampled , y_resampled = smote.fit_resample(X_train, y_train)

In [7]:
X_resampled.shape, y_resampled.shape

((190583, 30), (190583,))

In [8]:
# Create a Random Forest classifier
rf_classifier = RandomForestClassifier()
# Define the custom scoring function
def auprc_score(y_true, y_proba):
    precision, recall, _ = precision_recall_curve(y_true, y_proba)
    return auc(recall, precision)
custom_scorer = make_scorer(auprc_score, greater_is_better=True ,needs_proba=True)


In [9]:

# Define the hyperparameter search space
param_dist = {
    'n_estimators': randint(100, 1000),  # Random integer values between 100 and 1000
    'max_depth': randint(5, 20),          # Random integer values between 5 and 20
    'min_samples_split': randint(2, 10),  # Random integer values between 2 and 10
    'min_samples_leaf': randint(1, 10),   # Random integer values between 1 and 10
    'max_features': ['sqrt', 'log2'],     # List of options to choose from
    'bootstrap': [True, False]            # List of options to choose from
}

# Create a RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=rf_classifier,
    param_distributions=param_dist,
    n_iter=10,  # Number of parameter settings to sample
    cv=5,       # Number of cross-validation folds
    scoring=custom_scorer,  # Evaluation metric
    random_state=42      # Random state for reproducibility
)

# Perform the random search
random_search.fit(X_resampled, y_resampled)

# Print the best parameters and best score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)


Best Parameters: {'bootstrap': True, 'max_depth': 19, 'max_features': 'log2', 'min_samples_leaf': 3, 'min_samples_split': 5, 'n_estimators': 666}
Best Score: 0.9997859175997462
